In [1]:
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import time
model = YOLO("yolov8n.pt")
tracker = DeepSort(max_age=1)
cap = cv2.VideoCapture("traffic.mp4")
line_y = 300
total_count = set()
FRAME_SKIP = 2
SHOW_PREVIEW = True
frame_id = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_id += 1
    if frame_id % FRAME_SKIP != 0:
        continue
    results = model(frame, stream=True, verbose=False)
    detections = []
    for r in results:
        for box in r.boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            label = model.names[cls]
            if label in ["car", "bus", "truck"] and conf > 0.4:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                detections.append(([x1, y1, x2 - x1, y2 - y1], conf, label))
    tracks = tracker.update_tracks(detections, frame=frame)
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, w, h = track.to_ltwh()
        cx, cy = int(l + w / 2), int(t + h / 2)
        cv2.rectangle(frame, (int(l), int(t)), (int(l + w), int(t + h)), (0, 255, 0), 2)
        cv2.putText(frame, f"ID:{track_id}", (int(l), int(t - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        cv2.circle(frame, (cx, cy), 3, (0, 0, 255), -1)
        if cy > line_y and track_id not in total_count:
            total_count.add(track_id)
    cv2.line(frame, (0, line_y), (frame.shape[1], line_y), (0, 0, 255), 2)
    cv2.putText(frame, f"Cars Passed: {len(total_count)}", (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 4)
    if SHOW_PREVIEW:
        cv2.imshow("Real-Time Car Counter", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()
print(f"Total Car Passed: {len(total_count)}")

c:\Users\Deepesh_dala\AppData\Local\Programs\Python\Python313\Lib\site-packages\deep_sort_realtime\embedder\embedder_pytorch.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Total Car Passed: 12
